In [1]:
import numpy as np
import cvxpy as cvx
from qcqp import *
import scipy
import cvxpy as cp

In [2]:
n = 25
np.random.seed(1)

# Make adjacency matrix.
p = 0.2
W = np.asmatrix(np.random.uniform(low=0.0, high=1.0, size=(n, n)))
for i in range(n):
    W[i, i] = 1
    for j in range(i+1, n):
        W[j, i] = W[i, j]
W = np.array((W < p).astype(float))

x = cvx.Variable((n, 1))
# obj = 0.25*(cvx.sum_entries(W) - cvx.quad_form(x, W))
# cons = [cvx.square(x) == 1]


sni_f0 = (0.25 * scipy.sparse.csr_matrix(W), scipy.sparse.csc_matrix((n,1)), -0.25 * np.sum(W))
sni_fs = []
Qi = np.zeros((n, n))
for i  in range(n):
    Qi[i,i] = 1
    sni_fs += [(scipy.sparse.csr_matrix(Qi), scipy.sparse.csc_matrix((n, 1)), -1, "==")]
    Qi[i,i] = 0
prob = QCQPProblem(sni_f0, sni_fs, x)
qcqp = QCQP(prob)

# sample from the semidefinite relaxation
qcqp.suggest(SDR)
print("SDR-based upper bound: %.4f" % qcqp.sdr_bound)
f_cd, v_cd = qcqp.improve(COORD_DESCENT)
print("Coordinate descent: objective %.4f, violation %.4f" % (f_cd, v_cd))

qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(COORD_DESCENT)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Coordinate descent + ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

# SDR solution is cached and not solved again
qcqp.suggest(SDR)
f_dccp, v_dccp = qcqp.improve(DCCP, tau=1)
print("Penalty CCP: objective %.4f, violation %.4f" % (f_dccp, v_dccp))

qcqp.suggest(SDR)
f_admm, v_admm = qcqp.improve(ADMM)
print("Nonconvex ADMM: objective %.4f, violation %.4f" % (f_admm, v_admm))

SDR-based upper bound: -57.2072
Coordinate descent: objective -53.0004, violation 0.0001
SDR lower bound: -57.2072
Coordinate descent + ipopt: objective -54.0000, violation 0.0000
SDR lower bound: -57.2072
Ipopt: objective -53.0000, violation 0.0000
Penalty CCP: objective -53.0000, violation 0.0000
Nonconvex ADMM: objective -44.6927, violation 0.9832
